### Week 6, Day 2

We're about to create and use our own MCP Server and MCP Client!

It's pretty simple, but it's not super-simple. The excitment around MCP is about how easy it is to share and use other MCP Servers - making our own does involve a bit of work.

Let's review some python code made mostly by a hard-working Engineering Team:

accounts.py

In [1]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
from IPython.display import display, Markdown

load_dotenv(override=True)

True

In [2]:
from accounts import Account

In [3]:
account = Account.get("Ed")
account

Account(name='ed', balance=9969.94, strategy='', holdings={'AMZN': 3}, transactions=[3 shares of AMZN at 10.02 each.], portfolio_value_time_series=[('2025-08-16 12:44:38', 10161.94), ('2025-08-16 12:44:38', 10047.94)])

In [4]:
account.buy_shares("AMZN", 3, "Because this bookstore website looks promising")

'Completed. Latest details:\n{"name": "ed", "balance": 9762.526, "strategy": "", "holdings": {"AMZN": 6}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 10.02, "timestamp": "2025-08-16 12:44:38", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 69.138, "timestamp": "2025-08-16 12:47:37", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-08-16 12:44:38", 10161.94], ["2025-08-16 12:44:38", 10047.94], ["2025-08-16 12:47:37", 9852.526]], "total_portfolio_value": 9852.526, "total_profit_loss": -147.47400000000016}'

In [5]:
account.report()

'{"name": "ed", "balance": 9762.526, "strategy": "", "holdings": {"AMZN": 6}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 10.02, "timestamp": "2025-08-16 12:44:38", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 69.138, "timestamp": "2025-08-16 12:47:37", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-08-16 12:44:38", 10161.94], ["2025-08-16 12:44:38", 10047.94], ["2025-08-16 12:47:37", 9852.526], ["2025-08-16 12:47:37", 10086.526]], "total_portfolio_value": 10086.526, "total_profit_loss": 86.52599999999984}'

In [6]:
account.list_transactions()

[{'symbol': 'AMZN',
  'quantity': 3,
  'price': 10.02,
  'timestamp': '2025-08-16 12:44:38',
  'rationale': 'Because this bookstore website looks promising'},
 {'symbol': 'AMZN',
  'quantity': 3,
  'price': 69.138,
  'timestamp': '2025-08-16 12:47:37',
  'rationale': 'Because this bookstore website looks promising'}]

### Now we write an MCP server and use it directly!

In [14]:
# Now let's use our accounts server as an MCP server

params = {"command": "uv", "args": ["run", "accounts_server.py"]}
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
    mcp_tools = await server.list_tools()


CancelledError: 

In [ ]:
mcp_tools

In [8]:
instructions = "You are able to manage an account for a client, and answer questions about the account."
request = "My name is Ed and my account is under the name Ed. What's my balance and my holdings?"
# model = "gpt-4.1-mini"


In [ ]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
import os
from openai import OpenAI, AsyncOpenAI
from agents import Agent, OpenAIChatCompletionsModel, Runner, function_tool, set_tracing_disabled

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    client = AsyncOpenAI(
            api_key=os.getenv("GEMINI_API_KEY"),
            base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
        )

    model = OpenAIChatCompletionsModel(
        model="gemini-2.5-flash-lite",
        openai_client=client
    )
    agent = Agent(
        name="investigator", 
        instructions=instructions, 
        model=model,
        mcp_servers=[mcp_server]
        )
    
    with trace("account_manager"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))


### Now let's build our own MCP Client

In [11]:
from accounts_client import get_accounts_tools_openai, read_accounts_resource, list_accounts_tools

mcp_tools = await list_accounts_tools()
print(mcp_tools)
openai_tools = await get_accounts_tools_openai()
print(openai_tools)

CancelledError: 

In [ ]:
request = "My name is Ed and my account is under the name Ed. What's my balance?"

with trace("account_mcp_client"):
    agent = Agent(name="account_manager", instructions=instructions, model=model, tools=openai_tools)
    result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

In [ ]:
context = await read_accounts_resource("ed")
print(context)

In [ ]:
from accounts import Account
Account.get("ed").report()

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercises</h2>
            <span style="color:#ff7800;">Make your own MCP Server! Make a simple function to return the current Date, and expose it as a tool so that an Agent can tell you today's date.<br/>Harder optional exercise: then make an MCP Client, and use a native OpenAI call (without the Agents SDK) to use your tool via your client.
            </span>
        </td>
    </tr>
</table>